# LLM refinement with langchain

In [1]:
import os
import re
import json
import importlib
import chromadb
import pickle as pkl
import pandas as pd
from dotenv import dotenv_values
from langchain_chroma import Chroma
from typing import List, Optional
from pydantic import BaseModel, Field
from langchain.chat_models import ChatOpenAI
from langchain_voyageai import VoyageAIEmbeddings
from langchain_core.prompts import PromptTemplate
from langchain.chat_models import init_chat_model
from langchain.retrievers import BM25Retriever, EnsembleRetriever
from langchain_core.prompts import ChatPromptTemplate
from langchain_ollama import ChatOllama
import sys

sys.path.append('../src/')

/Users/malenadiazrio/Documents/UOC/TFM/TFM_2025_1/.venv/lib/python3.11/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


In [2]:
for i,v in dotenv_values().items():
    globals()[i]=v 

In [3]:
RESOURCES_DIR="../../resources"
llm = init_chat_model("o4-mini", model_provider="openai")
# llm = ChatOllama(model="qwen3:8b")
embeddings_model = VoyageAIEmbeddings(model="voyage-3")

Texto a probar

In [ ]:
test_df = pd.read_csv(os.path.join(PROJECT_DIR, "./datasets/TFM_test.csv"))
test_df.annotations = test_df.annotations.apply(eval)
clinical_note = test_df.texts.loc[1]
hpo_codes = test_df.annotations.iloc[1]
clinical_note

"Un hombre de 32 años acudió a una unidad regional de cirugía general por anemia sintomática grave. En un lapso de 4 meses, su hemoglobina descendió 62 puntos desde 137 gL (N: 135-180 gL). Su historial médico incluía epilepsia con valproato, arritmia cardiaca con flecainida, síndrome del túnel carpiano y compresión de la raíz nerviosa C5. No fuma, consume poco alcohol, tiene dos perros en casa y dos hijos. Sus antecedentes familiares incluían a su abuelo, que tuvo cáncer de intestino diagnosticado a los 78 años, y una abuela con cáncer de mama. La primera modalidad de diagnóstico por imagen fue un TAC que mostraba una gran masa fungiforme en el cuerpo del estómago que se extendía hasta el píloro y la primera parte del duodeno. Se sometió a una endoscopia digestiva alta (EDA) para investigar la patología. La histopatología arrojó con frecuencia un resultado de tejido hiperplásico benigno. Los primeros resultados histopatológicos informaron de mucosa gástrica con hiperplasia foveolar men

Definir el formato del input

In [4]:
chroma_client = chromadb.HttpClient(host='localhost', port=8001)

In [5]:
vectordb = Chroma(client=chroma_client, embedding_function=embeddings_model, 
                  collection_name="hpo_ontology_esp_FULL")
retriever = vectordb.as_retriever(search_kwargs={"k": 20})

In [ ]:
with open(os.path.join(PROJECT_DIR, "resources", "keyword_retriever.pkl"), "rb") as fp:
    keyword_retriever = pkl.load(fp)

In [20]:
vectordb.get_by_ids([
    "HP:0003812",
    "HP:0003745",
    "HP:0002671",
    "HP:0001263"
  ])

[Document(id='HP:0001263', metadata={'lineage': 'HP:0012758->HP:0012759->HP:0012638->HP:0000707->HP:0000118', 'hpo_id': 'HP:0001263'}, page_content='Retardo global del desarrollo. Retraso cognitivo. Retraso en el desarrollo cognitivo. Retraso en el desarrollo. Retraso en los hitos del desarrollo. Retraso en el desarrollo intelectual. Hitos retrasados. Retraso en el desarrollo psicomotor. Retraso en el desarrollo. Retraso del desarrollo en la primera infancia. Retraso global del desarrollo. Retraso en el desarrollo. GDD. Falta de desarrollo psicomotor. Retraso mental y motor. Retraso motor y del desarrollo. Retraso mental. Retraso psicomotor. Deficiencia en el desarrollo psicomotor. Fracaso del desarrollo psicomotor. Retraso del desarrollo psicomotor. Retraso en el desarrollo. Retraso en el desarrollo mental. Retraso en el desarrollo psicomotor. Retraso en la consecución de hitos motores o mentales en los ámbitos del desarrollo de un niño, incluidas las habilidades motoras, el habla y e

In [ ]:
with open(os.path.join(PROJECT_DIR, "./resources/keyword_retriever.pkl"), "rb") as fp:
    keyword_retriever = pkl.load(fp)

In [ ]:
ensemble_retriever = EnsembleRetriever(retrievers=[vectordb.as_retriever(),
                                                   keyword_retriever],
                                       weights=[0.6, 0.4], id_key="hpo_id", k=4)

In [ ]:
# prompt = """Revisa cuidadosamente cada frase de la nota clínica para identificar términos relacionados con patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos.
# Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. Incluye el contexto apropiado basándote únicamente en el pasaje.
# Devuelve los términos extraídos en un objeto JSON con una única clave 'fenotipos', que contiene la lista de términos extraídos escritos correctamente. Asegúrate de que el resultado sea conciso, sin notas, comentarios ni metaexplicaciones adicionales. No dejes fuera adjetivos críticos para ese fenotipo.
# <<NOTA CLÍNICA>>>
# {clinical_note}"""
# prompt = PromptTemplate.from_template(prompt)

Definir el output

In [15]:
system =  """Eres un experto de codificación de fenotipos de la ontología Human Phenotype Ontology. Para ello primero debes determinar qué fenotipos están presentes en la nota clínica. Sigue los siguientes pasos: 
1. A partir del siguiente texto clínico, identifica términos del texto que sugieran fenotipos clínicos relevantes, incluyendo diagnósticos, síntomas, signos físicos, hallazgos de laboratorio y modos de herencia.  
2. Si algún valor incluye de forma implícita un fenotipo, infiérelo y menciónalo como tal en el campo "phenotype".
3. Si el valor no permite inferir con seguridad un fenotipo, simplemente describe el resultado de la analítica en lenguaje natural.
4. Para cada término (extract), a parte del fenotipo, incluye la frase a la que pertenezca en la nota clínica original (context).
5. Sé específico, cada término debe contener un solo fenotipo asociado. Si tiene dos fenotipos, duplícalo y menciona ambos fenotipos. 
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{clinical_note}"),
    ]
)

In [7]:
class PhenotypeCandidate(BaseModel):
    """Fenotipos, patrones de herencia genética, anomalías anatómicas, síntomas clínicos, hallazgos diagnósticos, resultados de pruebas y afecciones o síndromes específicos en la nota clínica"""
    extract: str = Field(description="Un único fenotipo, diagnóstico, sintoma clínico, anomalia anatómica o prueba de laboratorio")
    phenotype: str = Field(description="Nombre del posible fenotipo asociado en español")
    context: str = Field( description="Deja esto vacío")


class Data(BaseModel):
    """Información extraída sobre los fenotipos encontrados y los términos y contexto en el que se encuentran"""
    candidates: List[PhenotypeCandidate]

In [16]:
structured_llm = llm.with_structured_output(Data)

In [17]:
extractphenotypes = prompt | structured_llm
answer = extractphenotypes.invoke(
    {
        "clinical_note": "Los pacientes que presentan schwannomas vestibulares unilaterales a una edad temprana o con características adicionales de neurofibromatosis de tipo 2 (NF2) corren el riesgo de desarrollar enfermedad bilateral y transmitir un riesgo de tumores neurogénicos a su descendencia. Hemos identificado 15 pacientes de una serie de 537 con schwannomas vestibulares unilaterales que también presentaban uno o más de los siguientes factores: otros tumores (10/15), características de NF2 (3/15) o antecedentes familiares de tumores neurogénicos (5/15). No se detectaron mutaciones de la línea germinal de la NF2 y en 7/9 casos en los que se disponía de material tumoral para el análisis se ha excluido una mutación de la línea germinal en el gen de la NF2. Aunque sigue existiendo la posibilidad de mosaicismo gonosómico, ahora es posible realizar pruebas de exclusión para la descendencia. Sugerimos una estrategia general, basada en el análisis del ADN tumoral, para distinguir los casos esporádicos y familiares de tumores causados por dos mecanismos de hit. La aplicación de esta estrategia sugiere que la mayoría de los casos de schwannoma vestibular unilateral que no cumplen los criterios de la NF2 son casuales.",
    }
)

In [18]:
answer.candidates

[PhenotypeCandidate(extract='schwannomas vestibulares unilaterales', phenotype='Schwannoma vestibular unilateral', context=''),
 PhenotypeCandidate(extract='a una edad temprana', phenotype='Inicio temprano de la enfermedad', context=''),
 PhenotypeCandidate(extract='características adicionales de neurofibromatosis de tipo 2 (NF2)', phenotype='Neurofibromatosis tipo 2', context=''),
 PhenotypeCandidate(extract='enfermedad bilateral', phenotype='Schwannoma vestibular bilateral', context=''),
 PhenotypeCandidate(extract='tumores neurogénicos a su descendencia', phenotype='Tumores neurogénicos hereditarios', context=''),
 PhenotypeCandidate(extract='otros tumores', phenotype='Tumores', context=''),
 PhenotypeCandidate(extract='antecedentes familiares de tumores neurogénicos', phenotype='Antecedentes familiares de tumores neurogénicos', context=''),
 PhenotypeCandidate(extract='No se detectaron mutaciones de la línea germinal de la NF2', phenotype='Ausencia de mutación germinal de NF2', con

In [ ]:
with open("../../resources/names_dict.pkl", "rb") as fp:
    names_dict = pkl.load(fp)

In [ ]:
from langchain_core.prompts import ChatPromptTemplate, HumanMessagePromptTemplate, SystemMessagePromptTemplate

sys_template = """Identifica el término de la Ontología de Fenotipos Humanos (HPO) más apropiado para cada extracto de las notas clínicas del paciente a partir de una lista de candidatos (Código HPO - Descripción).
Da prioridad a los términos que sean concisos y directamente pertinentes para el síntoma o afección principal descritos. 
Céntrate en el tema central de cada frase y evita seleccionar opciones con detalles descriptivos o situacionales adicionales a menos que sean esenciales para captar con precisión el fenotipo. 
Asegúrate de que el término HPO elegido coincide estrechamente con la afección del paciente tal como se describe, sin añadir términos nuevos o extraños. 
Si hay varios candidatos, selecciona y devuelve el término HPO más pertinente que mejor represente la afección o síntoma primario. Proporciona sólo los códigos HPO elegidos bajo la llave 'hpo_code' y con el formato 'HPXXXXXXX' donde la X es un número. La nota clínica original es la siguiente:
{clinical_note}
"""

human_template =  """Término: {term} ({phenotype})
Contexto: {context}
Candidatos: {candidates}"""

chat_template = ChatPromptTemplate.from_messages([
    ('system', sys_template),
    ('human', human_template)
])

# mesages = chat_template.format_messages(term="Bob", context="What is your name?", candidates="hey")

In [ ]:
class HpoCode(BaseModel):
    """Código HPO asignado con el formato HP:#######"""
    hpo_code: str

In [ ]:
llm_output = llm.with_structured_output(HpoCode)

In [ ]:
hpo_assignment = chat_template | llm_output

In [ ]:
from utils.fuzzyretriever import FuzzyRetriever
fuzzyretriever = FuzzyRetriever()

In [ ]:
fuzzyretriever.invoke("motora")

[]

In [ ]:
def pretty_print_candidates(docs):
    final_str = ""
    for doc in docs: 
        final_str += f"{doc.id} - {doc.page_content}\n"
    return final_str

In [ ]:
from langchain_core.runnables import chain
from langsmith import traceable

@chain
def custom_chain(question):
    response = extractphenotypes.invoke(question)
    docs = []
    intermediate_results = []
    for query in response.candidates:
        new_docs = retriever.invoke(query.extract)
        fuzzy_docs = [x[0] for x in fuzzyretriever.invoke(query.phenotype)]
        new_docs =  vectordb.get_by_ids(set(fuzzy_docs)) + new_docs if len(set(fuzzy_docs)) > 0 else new_docs
        docs.append({"clinical_note":question,
                     "term":query.extract,
                     "phenotype":query.phenotype,
                     "context":query.context,
                     "candidates":pretty_print_candidates(new_docs)})
        intermediate_results.append([doc.id for doc in new_docs])    
    answer = hpo_assignment.batch(docs)
    return {"final answer": answer, "docs": intermediate_results}

In [ ]:
import importlib
import utils.customchain as cc
custom_chain = cc.custom_chain

In [ ]:
from langfuse.callback import CallbackHandler
langfuse_handler = CallbackHandler(
    public_key="pk-lf-4c4b9492-d7ca-4f6d-b7af-325ea09726c0",
    secret_key="sk-lf-1b2e42c8-bf1e-4150-bf36-fce439ad2270",
    host="http://localhost:3000"
)

In [ ]:
answer = custom_chain.invoke({"clinical_note":clinical_note})

In [ ]:
answer = custom_chain.with_config(
    {
        "run_name": "qwen",
        "metadata": {"version": "v1", "owner": "mdiazrio"},
        "callbacks": [langfuse_handler]
    }
).invoke({"clinical_note":clinical_note})

In [ ]:
answer

{'final answer': [HpoCode(hpo_code='HP0001903'),
  HpoCode(hpo_code='HP0001903'),
  HpoCode(hpo_code='HP0007750'),
  HpoCode(hpo_code='HP0034884'),
  HpoCode(hpo_code='HP0005263'),
  HpoCode(hpo_code='HP0004394'),
  HpoCode(hpo_code='HP0004783'),
  HpoCode(hpo_code='HP0005231'),
  HpoCode(hpo_code='HP:0005202'),
  HpoCode(hpo_code='HP0033769'),
  HpoCode(hpo_code='HP0005202'),
  HpoCode(hpo_code='HP0005202'),
  HpoCode(hpo_code='HP0005202'),
  HpoCode(hpo_code='HP0004795'),
  HpoCode(hpo_code='HP0004783'),
  HpoCode(hpo_code='HP0012859'),
  HpoCode(hpo_code='HP0004784'),
  HpoCode(hpo_code='HP0004296'),
  HpoCode(hpo_code='HP0032222'),
  HpoCode(hpo_code='HP0004784'),
  HpoCode(hpo_code='HP0005238'),
  HpoCode(hpo_code='HP0001009'),
  HpoCode(hpo_code='HP0004784')],
 'docs': [['HP:0001903',
   'HP:0030784',
   'HP:0020060',
   'HP:0011895',
   'HP:0001903',
   'HP:0001895',
   'HP:0011031',
   'HP:0033264',
   'HP:0034738',
   'HP:0004863',
   'HP:0020059',
   'HP:0001908',
   'HP:0020

Raw GPT

In [ ]:
system = """Eres una herramienta que sirve para extraer fenotipos de la ontología Human Phenotype Ontolgy a partir de notas clínicas para ello: 
1. A partir del siguiente texto clínico, identifica todos los fenotipos clínicos relevantes, incluyendo diagnósticos, síntomas, signos físicos y hallazgos de laboratorio. 
2. Ignora por completo los hallazgos negativos, los hallazgos normales (es decir, «normal» o «no»), los procedimientos y los antecedentes familiares. 
3. Si algún valor incluye de forma implícita un fenotipo, infiérelo y menciónalo como tal.
4. Para cada término, asigna el código HPO apropiado. 
Devuelve un JSON con la llave "final_answer" y el listado de códigos HPO detectados.
"""
prompt = ChatPromptTemplate.from_messages(
    [
        ("system", system),
        ("human", "{clinical_note}"),
    ]
)

In [ ]:
import json
from langchain_core.runnables import chain

init_chain = prompt | llm

@chain
def rawgptchain(question):
    response = init_chain.invoke(question)
    return json.loads(response.content)

In [ ]:
import sys
sys.path.append('../utils')
import importlib
import rawgptchain as rgp

In [ ]:
importlib.reload(rgp)
rawgptchain = rgp.rawgptchain

In [ ]:
response = rawgptchain.with_config({"callbacks": [langfuse_handler]}).invoke({"clinical_note":clinical_note})

FlashRank

In [ ]:
from langchain.retrievers import ContextualCompressionRetriever
from langchain.retrievers.document_compressors import FlashrankRerank

In [ ]:
from flashrank import Ranker 

ranker = Ranker(model_name="ms-marco-MiniLM-L-12-v2")

In [ ]:
compressor = FlashrankRerank()
compression_retriever = ContextualCompressionRetriever(
    base_compressor=compressor, base_retriever=retriever
)

In [ ]:
compression_retriever.invoke(context)

[Document(metadata={'id': 8, 'relevance_score': 0.9918292, 'hpo_id': 'HP:0006780', 'lineage': 'HP:0100733->HP:0000818->HP:0011766->HP:0000828->HP:0100568->HP:0002664->HP:0011793->HP:0000118'}, page_content='Carcinoma de paratiroides. Tumor maligno de las glándulas paratiroides. El carcinoma paratiroideo suele segregar hormona paratiroidea, lo que provoca hiperparatiroidismo.'),
 Document(metadata={'id': 4, 'relevance_score': 0.78583723, 'hpo_id': 'HP:0100031', 'lineage': 'HP:0000820->HP:0000818->HP:0100568->HP:0002664->HP:0011793->HP:0000118->HP:0011772'}, page_content='Neoplasia del tiroides. Tumor (crecimiento anormal de tejido) de la glándula tiroides.'),
 Document(metadata={'id': 1, 'relevance_score': 0.60732955, 'hpo_id': 'HP:0011779', 'lineage': 'HP:0002890->HP:0100031->HP:0000820->HP:0000818->HP:0100568->HP:0002664->HP:0011793->HP:0000118->HP:0011772'}, page_content='Carcinoma anaplásico de tiroides.')]

In [ ]:
reranked_docs = []
for context in [c.context for c in answer.candidates]:
    reranked_docs.append(compression_retriever.invoke(context))

In [ ]:
reranked_docs

[[Document(metadata={'id': 5, 'relevance_score': 0.99929035, 'hpo_id': 'HP:0045081', 'lineage': 'HP:0004323->HP:0001507->HP:0000118'}, page_content='Anomalía del índice de masa corporal. Anomalía en la relación peso/altura al cuadrado, calculada dividiendo el peso del individuo en kilogramos por el cuadrado de la altura del individuo en metros y utilizada como indicador de obesidad e insuficiencia ponderal en comparación con las medias.'),
  Document(metadata={'id': 6, 'relevance_score': 0.99896955, 'hpo_id': 'HP:0031418', 'lineage': 'HP:0045081->HP:0004323->HP:0001507->HP:0000118'}, page_content='Aumento del índice de masa corporal. Relación peso/altura al cuadrado anormalmente elevada, calculada dividiendo el peso del individuo en kilogramos por el cuadrado de la altura del individuo en metros y utilizada como indicador de sobrepeso en comparación con las medias.'),
  Document(metadata={'id': 0, 'relevance_score': 0.99863136, 'hpo_id': 'HP:6000179', 'lineage': 'HP:0430103->HP:0032443